## Cargar y parsear archivo

In [3]:
import pandas as pd

### Conjuntos positivos

In [1]:
# Leer el archivo .txt, cada línea es una secuencia
with open("/content/pos_test_ds1.txt", "r") as f:
    # elimina saltos de línea vacíos
    secuencias = [line.strip() for line in f if line.strip()]

In [4]:
# Crear el dataframe con dos columnas: 'sequence' y 'label'
df_pos1 = pd.DataFrame({
    "sequence": secuencias,
    "label": [1] * len(secuencias)  # asigna 1 a todas las filas
})

In [5]:
display(df_pos1.head())

,sequence,label
0,SEKDEL,1
1,EDPYRFFE,1
2,ENCGRQAG,1
3,DEKGPKWKR,1
4,QCIRLEKAR,1


In [6]:
df_pos1.shape

(291, 2)

In [7]:
# Leer el archivo .txt, cada línea es una secuencia
with open("/content/pos_train_ds1.txt", "r") as f:
    # elimina saltos de línea vacíos
    secuencias = [line.strip() for line in f if line.strip()]

In [8]:
# Crear el dataframe con dos columnas: 'sequence' y 'label'
df_pos2 = pd.DataFrame({
    "sequence": secuencias,
    "label": [1] * len(secuencias)  # asigna 1 a todas las filas
})

In [9]:
display(df_pos2.head())

,sequence,label
0,HDEF,1
1,HDEL,1
2,KDEL,1
3,RWRW,1
4,AHKCIC,1


In [10]:
df_pos2.shape

(1168, 2)

### Conjuntos negativos

In [14]:
# Leer el archivo .txt, cada línea es una secuencia
with open("/content/neg_test_ds1.txt", "r") as f:
    # elimina saltos de línea vacíos
    secuencias = [line.strip() for line in f if line.strip()]

In [15]:
# Crear el dataframe con dos columnas: 'sequence' y 'label'
df_neg1 = pd.DataFrame({
    "sequence": secuencias,
    "label": [0] * len(secuencias)  # asigna 1 a todas las filas
})

In [16]:
display(df_neg1.head())

,sequence,label
0,KWRRWI,0
1,EAESEAEP,0
2,RIWVIWRK,0
3,KFWWWNYLK,0
4,RINPKIYRS,0


In [17]:
df_neg1.shape

(291, 2)

In [18]:
# Leer el archivo .txt, cada línea es una secuencia
with open("/content/neg_train_ds1.txt", "r") as f:
    # elimina saltos de línea vacíos
    secuencias = [line.strip() for line in f if line.strip()]

In [19]:
# Crear el dataframe con dos columnas: 'sequence' y 'label'
df_neg2 = pd.DataFrame({
    "sequence": secuencias,
    "label": [0] * len(secuencias)  # asigna 1 a todas las filas
})

In [20]:
display(df_neg2.head())

,sequence,label
0,PRRRK,0
1,EFKRIV,0
2,PGLGFY,0
3,RPQQVP,0
4,VVYWLL,0


In [21]:
df_neg2.shape

(1168, 2)

## Concatenar datasets

In [23]:
df = pd.concat([df_pos1, df_pos2, df_neg1, df_neg2], ignore_index=True)
display(df)

,sequence,label
0,SEKDEL,1
1,EDPYRFFE,1
2,ENCGRQAG,1
3,DEKGPKWKR,1
4,QCIRLEKAR,1
...,...,...
2913,MSYGFRFYDANGNVTVDSTNKSFRSVYRQQVFPVLGGTTNLPPGFD...,0
2914,QEGCEWESRPCNRGCWLPRQWESVHMRFGVLVLWRAPQFEHFRECC...,0
2915,MAEILKDFYAMNLFIFLIILPAKIRGETLSLTHPKCHHIMLPSLFI...,0
2916,MAVAKPPLQTAAVLLLLLLVVAAASWLQTVDAASGFCSSKCSVRCG...,0


In [24]:
df.shape

(2918, 2)

## Verificar duplicados

In [25]:
df["sequence"].unique().shape

(2918,)

In [26]:
df = df.drop_duplicates().copy()

In [27]:
df.shape

(2918, 2)

## Verificar canónicos

In [28]:
# Definir aminoácidos canónicos
aminoacidos = set("ACDEFGHIKLMNPQRSTVWY")

In [29]:
# Verificar cuáles secuencias válidas
df["is_valid"] = [set(seq).issubset(aminoacidos) for seq in df["sequence"]]

In [30]:
no_validas = df[~df["is_valid"]]
print("Secuencias inválidas:", len(no_validas))

Secuencias inválidas: 0


In [31]:
df_validas = df[df["is_valid"]].drop(columns="is_valid")
print("Secuencias válidas restantes:", len(df_validas))

Secuencias válidas restantes: 2918


In [32]:
df_validas.to_csv('PeptideReactor_labeled.csv', index=False, header=True)

## Obtener metadatos

In [33]:
import json

In [34]:
def export_json(path_to_export, data_to_export):
    with open(path_to_export, 'w') as doc_export:
        json.dump(
            data_to_export,
            doc_export,
            indent=4,
            default=str,
            ensure_ascii=False)

In [35]:
def create_metada_with_multiple_values(df_metada_filter, full_df):
    dict_metadata = {}

    for column in df_metada_filter.columns:
        values = df_metada_filter[column].unique().tolist()

        if len(values)>1:
            values = [str(value) for value in values]
            values = ";".join(values)
            dict_metadata.update({column:values})
        else:
            dict_metadata.update({column:values[0]})

    dict_metadata.update({
        "number_of_sequences" : len(full_df)
    })

    return dict_metadata

In [36]:
def read_metadata(path_data, name_source):
    df_metada = pd.read_excel(path_data)
    df_metada_filter = df_metada[df_metada["name source"] == name_source]
    df_metada_filter = df_metada_filter[['type source',
                                         'estatico-dinamico',
                                         'licencia',
                                         'año de publicación',
                                         'fecha ultima actualizacion',
                                         'download date',
                                         'formato',
                                         'peptide property',
                                         'informacion del dataset',
                                         'unidad de medida',
                                         'Construccion de dataset negativos',
                                         'repositorio o servidor',
                                         'Publicacion',]]
    return df_metada_filter

In [37]:
df_metada_filter = read_metadata("/content/description_raw_data.xlsx", "peptidereactor")

In [38]:
dict_metadata = create_metada_with_multiple_values(df_metada_filter, df)

In [39]:
export_json("metadata.json", dict_metadata)